In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
import numpy as np
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [2]:
# Set paths for AI and real images
ai_data_path = "/"""give your path here""""""/ "      # Path to AI-generated images
real_data_path = "/"""give your path here""""""/"  # Path to real images

In [3]:
# Load and preprocess image data 
def load_images(ai_data_path, real_data_path, img_size=(224, 224)):
    images = []
    labels = []
    
    # Load real images and label them as 0
    for img_file in os.listdir(real_data_path):
        img_path = os.path.join(real_data_path, img_file)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        images.append(img_array)
        labels.append(0)
    
    # Load AI-generated images and label them as 1
    for img_file in os.listdir(ai_data_path):
        img_path = os.path.join(ai_data_path, img_file)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        images.append(img_array)
        labels.append(1)
    
    return np.array(images), np.array(labels)


In [4]:
# Load images and labels
X, y = load_images(ai_data_path, real_data_path)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [5]:
# Build CNN model using VGG16 backbone for binary classification
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification layer with sigmoid activation

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the VGG16 base layers
for layer in base_model.layers:
    layer.trainable = False
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
# Training parameters
batch_size = 32
epochs = 20

In [8]:
# Training callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                    batch_size=batch_size, epochs=epochs, 
                    callbacks=[early_stopping, lr_scheduler])

Epoch 1/20
73/73 [==============================] - 302s 4s/step - loss: 0.2517 - accuracy: 0.8953 - val_loss: 0.4576 - val_accuracy: 0.7362 - lr: 0.0010
Epoch 2/20
73/73 [==============================] - 335s 5s/step - loss: 0.1313 - accuracy: 0.9470 - val_loss: 0.2685 - val_accuracy: 0.9310 - lr: 0.0010
Epoch 3/20
73/73 [==============================] - 320s 4s/step - loss: 0.1047 - accuracy: 0.9565 - val_loss: 0.1534 - val_accuracy: 0.9672 - lr: 0.0010
Epoch 4/20
73/73 [==============================] - 325s 4s/step - loss: 0.0810 - accuracy: 0.9698 - val_loss: 0.1010 - val_accuracy: 0.9741 - lr: 0.0010
Epoch 5/20
73/73 [==============================] - 323s 4s/step - loss: 0.0933 - accuracy: 0.9638 - val_loss: 0.0746 - val_accuracy: 0.9793 - lr: 0.0010
Epoch 6/20
73/73 [==============================] - 321s 4s/step - loss: 0.0887 - accuracy: 0.9638 - val_loss: 0.0586 - val_accuracy: 0.9828 - lr: 0.0010
Epoch 7/20
73/73 [==============================] - 321s 4s/step - loss: 0.0

In [9]:
# Save the trained model
model.save('Image_part.h5')

C:\Users\Radheshyam\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
# Load the saved model
model = tf.keras.models.load_model('Image_part.h5')


In [4]:
# Function to predict and classify the image
def predict_image(model, image_path):
    # Load and preprocess the image in one step
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension for prediction

    # Make prediction
    prediction = model.predict(image)[0][0]  # Get the prediction value directly
    label = "AI-generated" if prediction >= 0.5 else "Real"
    print(f"The image is predicted to be: {label}")


In [5]:
# Usage example
predict_image(model, 'testImage.jpg')

1/1 [==============================] - 1s 807ms/step
The image is predicted to be: Real


In [6]:
# Usage example
predict_image(model, 'AiImage.png')

1/1 [==============================] - 0s 173ms/step
The image is predicted to be: AI-generated
